In [ ]:
import os
import dotenv
import autogen
import subprocess
import sys

dotenv.load_dotenv()

# Ensure environment variables are loaded correctly
gpt_model = "gpt-4o"
api_key = os.getenv("OPENAI_API_KEY")
base_url = os.getenv("AZURE_OPENAI_ENDPOINT")
api_version = os.getenv("OPENAI_API_VERSION")

config_list = [
    {
        "model": gpt_model,
        "api_key": os.getenv("OPENAI_API_KEY"),
        "base_url": os.getenv("AZURE_OPENAI_ENDPOINT"),
        "api_version": os.getenv("OPENAI_API_VERSION"),
        "api_type": "azure",
    }
]

llm_config = {
    "cache_seed": 1,
    "temperature": 0,
    "config_list": config_list,
    "timeout": 120
}

def install_and_import(package):
    try:
        __import__(package)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        __import__(package)

In [ ]:
user_proxy = autogen.UserProxyAgent(
    name="User",
    system_message = "Executor. Execute the code written by the coder and suggest updates if there are errors.",
    human_input_mode = "NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "code",
        "use_docker": False,
    },
)

coder = autogen.AssistantAgent(
    name="Coder",
    llm_config = llm_config,
    system_message="""Coder. Your job is to write complete code. You primarily are a game programmer. Make sure to save the code to disk in the work directory. 
                    If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line.
                    """,
)

pm = autogen.AssistantAgent(
    name="Product_manager",
    system_message="Help plan out to create games. If should not write the code, but can provide guidance on how to structure the code and the main focus should be on the requirements mentioned by the user.",
    llm_config=llm_config,
)

group_chat = autogen.GroupChat(
    agents=[user_proxy, coder, pm], messages=[], max_round=15
)
manager = autogen.GroupChatManager(groupchat=group_chat, llm_config=llm_config)

user_proxy.initiate_chat(
    manager,
    message=
    """
    I would like to create a space invader game in Python!  Make sure the game ends when the player gets hit by the enemy red lasers.
    Make sure the game should have a score counter that increases by 1 each time the player hits an enery with blue lasers.
    The enemies should move from left to right and right to left on the screen and also shoot red lasers at the player randomly.
    Make sure that when the enemy laser (Red lasers) hit the player, the game should display a message saying "Game Over".
    When all the enemies are destroyed, the game should display a message saying "You Win" and display the final score.
    The "Game Over" page, should display the final score along with a restart button to restart the game and a quit button to quit the game.
    When the player hits the restart button, the game should restart with initial number of enemies at inital level and the score should be reset to 0.
    """,
)